<a href="https://colab.research.google.com/github/eswens13/style_transfer/blob/dev/erik/music_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before Running

Make sure you have changed the runtime environment to Python 2.7 and acceleration type to GPU.

In [1]:
# Colab has CUDA 9.2 installed by default. Magenta wants version 9.0.
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install -y cuda-9-0

--2018-11-24 23:27:05--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?bnayZj96OBu8qwHYa0fnpbzVVj011ibZ2lh8rsZ5kyStRz82Hlt50yb9uKKaJVJtXh3YG-tn0zb1rlA1vZxjXy5RWvnnxGBjRN8_O0nKyXoQLDAoHABmDgCpfOqAfZXfCJ1gAxjDilr2xx8AQpSEAMl3RUE_XE5zaU4fm-HIPxpnnAWx2Ow4tC9lIm1miz00YBhoT6yK2RgGouPNBsf1 [following]
--2018-11-24 23:27:06--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?bnayZj96OBu8qwHYa0fnpbzVVj011ibZ2lh8rsZ5kyStRz82Hlt50yb9uKKaJVJtXh3YG-tn0zb1rlA1vZxjXy5RWvnnxGBjRN8_O0nKyXoQLD

In [5]:
# Modify the CUDA symlink to point to the 9.0 version.
!unlink /usr/local/cuda
!ln -sf /usr/local/cuda-9.0 /usr/local/cuda
print "\nCheck that /usr/local/cuda is pointing to cuda 9.0:\n"
!ls -la /usr/local/


Check that /usr/local/cuda is pointing to cuda 9.0:

total 60
drwxr-xr-x  1 root root 4096 Nov 24 23:32 .
drwxr-xr-x  1 root root 4096 Nov 24 23:30 ..
drwxr-xr-x  1 root root 4096 Nov 20 18:37 bin
lrwxrwxrwx  1 root root   19 Nov 24 23:32 cuda -> /usr/local/cuda-9.0
drwxr-xr-x 15 root root 4096 Nov 24 23:30 cuda-9.0
drwxr-xr-x  1 root root 4096 Nov 20 18:05 cuda-9.2
drwxr-xr-x  1 root root 4096 Nov 20 18:16 etc
drwxr-xr-x  2 root root 4096 Oct 18 21:02 games
drwxr-xr-x  2 root root 4096 Oct 18 21:02 include
drwxr-xr-x  1 root root 4096 Nov 20 18:05 lib
lrwxrwxrwx  1 root root    9 Oct 18 21:02 man -> share/man
drwxr-xr-x  2 root root 4096 Oct 18 21:03 sbin
drwxr-xr-x  1 root root 4096 Nov 20 18:08 share
drwxr-xr-x  2 root root 4096 Oct 18 21:02 src


In [6]:
# These are some packages that need to be manually installed for rtmidi
!apt-get install build-essential libasound2-dev libjack-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 cuda-drivers : Depends: nvidia-396 (>= 396.44) but it is not going to be installed
 libcuda1-396 : Depends: nvidia-396 (>= 396.44) but it is not going to be installed
 libjack-dev : Depends: libjack0 (= 1:0.125.0-3) but it is not going to be installed
               Depends: uuid-dev but it is not going to be installed
 nvidia-396-dev : Depends: nvidia-396 (>= 396.44) but it is not going to be installed
 nvidia-opencl-icd-396 : Depends: nvidia-396 (>= 396.44) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).


In [7]:
# Install the GPU version of magenta.
!pip install magenta-gpu

    100% |████████████████████████████████| 1.1MB 14.7MB/s 
    100% |████████████████████████████████| 5.6MB 5.5MB/s 
    100% |████████████████████████████████| 204kB 23.4MB/s 
    100% |████████████████████████████████| 1.6MB 15.5MB/s 
    100% |████████████████████████████████| 281.7MB 81kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5569bf88c000 @  0x7f2f61a002a4 0x55696e85bf18 0x55696e94fa85 0x55696e86f4ca 0x55696e874232 0x55696e86cd0a 0x55696e8745fe 0x55696e86cd0a 0x55696e8745fe 0x55696e86cd0a 0x55696e8745fe 0x55696e86cd0a 0x55696e874c38 0x55696e86cd0a 0x55696e8745fe 0x55696e86cd0a 0x55696e8745fe 0x55696e874232 0x55696e874232 0x55696e86cd0a 0x55696e874c38 0x55696e874232 0x55696e86cd0a 0x55696e874c38 0x55696e86cd0a 0x55696e874c38 0x55696e86cd0a 0x55696e8745fe 0x55696e86cd0a 0x55696e86c629 0x55696e89d61f
    100% |████████████████████████████████| 71kB 23.5MB/s 
    100% |████████████████████████████████| 583kB 3.1MB/s 
    100% |████████████████████████████████| 16.1MB 1.3MB/s

In [0]:
# Will fail without the newer version of this package.
!pip install tensorflow-probability-gpu==0.5.0rc0

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
# We have to include the CUDA toolkit in the LD_LIBRARY_PATH environment
# variable. For some reason, the change has no effect when using the normal bash
# export syntax. Have to use python os.environ object.
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ['LD_LIBRARY_PATH']
print "The path below should include /usr/local/cuda/lib64"
!echo $LD_LIBRARY_PATH

The path below should include /usr/local/cuda/lib64
/usr/local/cuda/lib64:/usr/lib64-nvidia


In [17]:
# TODO: Should only run this command if the file doesn't exist to save on time.

# This command generates a '.tfrecord' file of NoteSequence objects from
# Beethoven midi files.
!convert_dir_to_note_sequences \
  --input_dir="/content/drive/My Drive/deep_learning/midis/beethoven" \
  --output_file="/content/drive/My Drive/deep_learning/notesequences.tfrecord" \
  --recursive

/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio4.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio_utils import squeeze_element, chars_to_strings
/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio5.py:98: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio5_utils import VarReader5
/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary in

In [19]:
# TODO: Should only run this command if the file doesn't exist to save on time.

# Options for the config argument are listed here:
#    https://github.com/tensorflow/magenta/tree/master/magenta/models/performance_rnn#pre-trained
# Assuming that just 'performance' is the simplest, for now.

# This command generates training data set comprised of SequenceExample objects.
# The NoteSequence objects that were built in the previous command somehow
# inform this process.
#
# NOTE: This command takes a LONG time!
!performance_rnn_create_dataset \
--config=performance \
--input="/content/drive/My Drive/deep_learning/notesequences.tfrecord" \
--output_dir="/content/drive/My Drive/deep_learning/performance_rnn/sequence_examples" \
--eval_ratio=0.10

/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio4.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio_utils import squeeze_element, chars_to_strings
/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio5.py:98: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio5_utils import VarReader5
/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary in

In [21]:
# Run one training step to confirm that it doesn't error out and to demonstrate
# GPU usage.
#
# TODO: Change the logdir to use date/time.
!performance_rnn_train \
--config=performance \
--run_dir="/content/drive/My Drive/deep_learning/performance_rnn/logdir/training_1" \
--sequence_example_file="/content/drive/My Drive/deep_learning/performance_rnn/sequence_examples/training_performances.tfrecord" \
--num_training_steps=1

/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio4.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio_utils import squeeze_element, chars_to_strings
/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio5.py:98: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .mio5_utils import VarReader5
/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary in

In [0]:
# TODO: For later.

## View stats on TensorBoard.
#tensorboard --logdir=/tmp/performance_rnn/logdir

## Run an eval job (I think forward pass only).
#performance_rnn_train \
#--config=${CONFIG} \
#--run_dir=/tmp/performance_rnn/logdir/run1 \
#--sequence_example_file=/tmp/performance_rnn/sequence_examples/eval_performances.tfrecord \
#--eval